In [1]:
import sys
sys.path.append("../")

import pandas as pd

from desci_sense.schema.notion_ontology_base import NotionOntologyBase, load_ontology_from_config
from desci_sense.shared_functions.schema.ontology_base import NotionOntologyConfig

In [2]:
notion_config = NotionOntologyConfig()
notion_config

NotionOntologyConfig(db_id=None, notion_api_token=None, versions=['v0'])

In [3]:
notion_ontology = load_ontology_from_config(notion_config)

In [4]:
df = notion_ontology.ont_df
df.head()

,name,display_name,uri,label,prompt,notes,valid_subject_types,valid_object_types,versions
name,,,,,,,,,
endorses,endorses,➕ endorses,None,endorses,this post endorses the mentioned reference. Th...,None,"[post, ref]",[ref],[v0]
disagreesWith,disagreesWith,👎 disagrees-with,http://purl.org/spar/cito/disagreesWith,disagrees,this post disputes or expresses disagreement w...,None,"[post, ref]",[ref],[v0]
agreesWith,agreesWith,👍 agrees-with,http://purl.org/spar/cito/agreesWith,agrees,"this post expresses agreement with statements,...",None,"[post, ref]",[ref],[v0]
possibleMissingReference,possibleMissingReference,⬛ possible-missing-reference,None,missing-ref,this post seems to be referring to a reference...,None,[post],[nan],[v0]
dgObservation,dgObservation,🔭 discourse-graph/observation,None,dg-observation,"this post is articulating a single, highly obs...",None,[post],[nan],[v0]


In [5]:
records = df.to_dict(orient="records")
records

[{'name': 'endorses',
  'display_name': '➕\xa0endorses',
  'uri': None,
  'label': 'endorses',
  'prompt': 'this post endorses the mentioned reference. This label can also be used for cases of implicit recommendation, where the author is expressing enjoyment of some content but not explicitly recommending it.',
  'notes': None,
  'valid_subject_types': ['post', 'ref'],
  'valid_object_types': ['ref'],
  'versions': ['v0']},
 {'name': 'disagreesWith',
  'display_name': '👎 disagrees-with',
  'uri': 'http://purl.org/spar/cito/disagreesWith',
  'label': 'disagrees',
  'prompt': 'this post disputes or expresses disagreement with statements, ideas or conclusions presented in the mentioned reference.',
  'notes': None,
  'valid_subject_types': ['post', 'ref'],
  'valid_object_types': ['ref'],
  'versions': ['v0']},
 {'name': 'agreesWith',
  'display_name': '👍 agrees-with',
  'uri': 'http://purl.org/spar/cito/agreesWith',
  'label': 'agrees',
  'prompt': 'this post expresses agreement with sta

In [29]:
from jinja2 import Environment, BaseLoader

def render_to_py_dict(obj_dict, obj_name: str = "object", out_path: str = "output.py"):
    template_str = '''{{ obj_name }} = {
    {% for key, value in obj_dict.items() -%}
    '{{ key }}': {{ value|to_py }},
    {% endfor -%}
}'''

    def to_py_filter(value):
        """Custom Jinja2 filter to convert Python objects to string representations, including nested dictionaries."""
        if isinstance(value, str):
            return '"' + value.replace('"', '\\"') + '"'
        elif value is None:
            return 'None'
        elif isinstance(value, list):
            return '[' + ', '.join(to_py_filter(v) for v in value) + ']'
        elif isinstance(value, dict):
            # Handle nested dictionaries
            dict_items = ', '.join(f"'{k}': {to_py_filter(v)}" for k, v in value.items())
            return '{' + dict_items + '}'
        elif isinstance(value, (int, float)):
            return str(value)
        else:
            raise TypeError(f"Unsupported type: {type(value)}")

    env = Environment(loader=BaseLoader())
    env.filters['to_py'] = to_py_filter

    template = env.from_string(template_str)
    rendered_content = template.render(obj_dict=obj_dict,
                                       obj_name=obj_name)
    
    with open(out_path, 'w') as file:
        file.write(rendered_content)

# Example usage with a complex nested dictionary
d = {
    "key_1": [
        {"a": [1, 2]}
    ],
    "key_2": {"b": 2, "c": 3}
}

render_to_py_dict(d, "ontology")

In [6]:
from typing import List, Dict, Union
import pandas as pd
from pydantic import Field, BaseModel
from pydantic_settings import BaseSettings, SettingsConfigDict

class OntologyPredicateDefinition(BaseModel):
    name: str = Field(description="Predicate name.")
    uri: Union[str,None] = Field(description="Linked data URI for this predicate.")
    versions: List[str] = Field(
        description="Which ontology versions is this item included in."
    )


class LLMOntologyPredicateDefinition(OntologyPredicateDefinition):
    label: str = Field(description="Output label model should use for this predicate")
    display_name: str = Field(description="Name to display in app front-ends.")
    prompt: str = Field(description="Description to use in prompt for this predicate")
    valid_subject_types: List[str] = Field(
        description="List of valid subject entity types for this predicate"
    )
    valid_object_types: List[str] = Field(
        description="List of valid object entity types for this predicate"
    )


In [7]:
preds = [LLMOntologyPredicateDefinition(**d) for d in records]

In [13]:
keyword_pred = OntologyPredicateDefinition(name="keyword", uri=None, versions=["v0"])

In [31]:
class KeywordPredicateDefinition(OntologyPredicateDefinition):
    """
    Type of OntologyPredicateDefinition with default initialization values 
    for each field.
    """
    name: str = Field("hasKeyword", description="Predicate name.")
    uri: str = Field("https://pcp-on-web.de/ontology/0.2/index-en.html#hasKeyword", description="Linked data URI for this predicate.")
    versions: List[str] = Field(["v0"], description="Which ontology versions is this item included in.")

# Example usage
keyword_predicate = KeywordPredicateDefinition()
print(keyword_predicate)

name='hasKeyword' uri='https://pcp-on-web.de/ontology/0.2/index-en.html#hasKeyword' versions=['v0']


In [ ]:
class OntologyInterface(BaseModel):
    semantic_predicates: List[LLMOntologyPredicateDefinition]
    keyword_predicate: KeywordPredicateDefinition = Field(default_factory=KeywordPredicateDefinition)
    ontology_config: NotionOntologyConfig = Field(default_factory=NotionOntologyConfig)

In [19]:
keyword_pred.model_dump()

{'name': 'keyword', 'uri': None, 'versions': ['v0']}

In [15]:
ontology = OntologyInterface(semantic_predicates=preds, keyword_predicate=keyword_pred)

In [17]:
ont_dict = ontology.model_dump()
ont_dict

{'semantic_predicates': [{'name': 'endorses',
   'uri': None,
   'versions': ['v0'],
   'label': 'endorses',
   'display_name': '➕\xa0endorses',
   'prompt': 'this post endorses the mentioned reference. This label can also be used for cases of implicit recommendation, where the author is expressing enjoyment of some content but not explicitly recommending it.',
   'valid_subject_types': ['post', 'ref'],
   'valid_object_types': ['ref']},
  {'name': 'disagreesWith',
   'uri': 'http://purl.org/spar/cito/disagreesWith',
   'versions': ['v0'],
   'label': 'disagrees',
   'display_name': '👎 disagrees-with',
   'prompt': 'this post disputes or expresses disagreement with statements, ideas or conclusions presented in the mentioned reference.',
   'valid_subject_types': ['post', 'ref'],
   'valid_object_types': ['ref']},
  {'name': 'agreesWith',
   'uri': 'http://purl.org/spar/cito/agreesWith',
   'versions': ['v0'],
   'label': 'agrees',
   'display_name': '👍 agrees-with',
   'prompt': 'this 

In [18]:
new_ont = OntologyInterface.model_validate(ont_dict)

In [25]:
render_to_py_dict(ont_dict)

In [11]:
p = preds[0]
p.model_dump()


{'name': 'endorses',
 'uri': None,
 'versions': ['v0'],
 'label': 'endorses',
 'display_name': '➕\xa0endorses',
 'prompt': 'this post endorses the mentioned reference. This label can also be used for cases of implicit recommendation, where the author is expressing enjoyment of some content but not explicitly recommending it.',
 'valid_subject_types': ['post', 'ref'],
 'valid_object_types': ['ref']}

In [ ]:
d = {
    "key_1": [
        {"a": [1,2]}
    ],
    "key_2": {"b": 2, "c": 3}

}

In [31]:
"reviews" in notion_ontology.dis

AttributeError: 'NotionOntologyBase' object has no attribute 'display_name_df'

In [24]:
ont_df.display_name.to_list()

['⬛ possible-missing-reference',
 '🔭 discourse-graph/observation',
 '🫴 discourse-graph/claim',
 ' ❓ discourse-graph/question',
 '👀 watching-status',
 '📑 reading-status',
 '🎧 listening-status',
 '🔗 links-to',
 '🧐 reviews',
 '👌 recommends',
 '❔ ask-question-about',
 '📝 quotes-from',
 '🗣️ discusses',
 '🗓️ announces-event',
 '📢 announces-job',
 '📢 announces-research']

In [21]:
label_key_df = ont_df.set_index("label")
label_key_df.loc["missing-ref"]["display_name"]

'⬛ possible-missing-reference'

In [27]:
ont_df.to_dict(orient='index')

{'possible-missing-reference': {'display_name': '⬛ possible-missing-reference',
  'URI': None,
  'label': 'missing-ref',
  'prompt': 'this post seems to be referring to a reference by name but has not explicitly provided a URL link to the reference. For example, a post that discusses a book and mentions it by title, but contains no link to the book.',
  'notes': None,
  'valid_subject_types': ['post'],
  'valid_object_types': ['nan'],
  'versions': ['v0']},
 'dg-observation': {'display_name': '🔭 discourse-graph/observation',
  'URI': None,
  'label': 'dg-observation',
  'prompt': 'this post is articulating a single, highly observation. The intuition is that observation notes should be as close to “the data” as possible. They should be similar to how results are described in results sections of academic publications.',
  'notes': None,
  'valid_subject_types': ['post'],
  'valid_object_types': ['nan'],
  'versions': ['v0']},
 'dg-claim': {'display_name': '🫴 discourse-graph/claim',
  'UR

In [4]:
notion_ontology.get_valid_templates(subject_type="post", object_type="ref")

[{'display_name': '👀 watching-status',
  'URI': None,
  'label': 'watching',
  'prompt': 'this post describes the watching status of the author in relation to a reference, such as a video or movie. The author may have watched the content in the past, is watching the content in the present, or is looking forward to watching the content in the future.',
  'notes': None,
  'valid_subject_types': ['post'],
  'valid_object_types': ['ref'],
  'versions': ['v0']},
 {'display_name': '📑 reading-status',
  'URI': None,
  'label': 'reading',
  'prompt': 'this post describes the reading status of the author in relation to a reference, such as a book or article. The author may either have read the reference in the past, is reading the reference in the present, or is looking forward to reading the reference in the future.',
  'notes': None,
  'valid_subject_types': ['post'],
  'valid_object_types': ['ref'],
  'versions': ['v0']},
 {'display_name': '🎧 listening-status',
  'URI': None,
  'label': 'lis

In [8]:
notion_ontology.get_valid_templates(subject_type="post", object_type="ref", as_dict=False).label.to_list()

['watching',
 'reading',
 'listening',
 'other',
 'review',
 'recommendation',
 'question',
 'quote',
 'discussion',
 'event',
 'job',
 'announce']

In [9]:
notion_ontology.get_valid_templates(subject_type="post", object_type="nan", as_dict=False).label.to_list()

['missing-ref', 'dg-observation', 'dg-claim', 'dg-question', 'other']

In [5]:
notion_ontology.get_valid_templates(subject_type="ref", object_type="ref")

[{'display_name': '🧐 reviews',
  'URI': None,
  'label': 'review',
  'prompt': 'this post contains a review of another reference, such as a book, article or movie. The review could be positive or negative. A review can be detailed or a simple short endorsement.',
  'notes': None,
  'valid_subject_types': ['post', 'ref'],
  'valid_object_types': ['ref'],
  'versions': ['v0']},
 {'display_name': '👌 recommends',
  'URI': None,
  'label': 'recommendation',
  'prompt': 'The author is recommending any kind of content: an article, a movie, podcast, book, another post, etc. This tag can also be used for cases of implicit recommendation, where the author is expressing enjoyment of some content but not explicitly recommending it.',
  'notes': None,
  'valid_subject_types': ['post', 'ref'],
  'valid_object_types': ['ref'],
  'versions': ['v0']},
 {'display_name': '❔ ask-question-about',
  'URI': None,
  'label': 'question',
  'prompt': "this post is raising a question or questions about some cont